In [1]:
%reload_ext autoreload
%autoreload 2

import optuna

from torchinfo import summary

# import import_ipynb

c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


##### Hyperparameter Tuning with Accuracy Pruning


In [3]:
import models_hyper_optim_optuna

DEVICE found:  cuda:0
c:\Users\androgo\Documents\Python Scripts\IDS\CICIDS2017_notebooks\IDS_Project
X_train__78_features (500, 78)
X_test__78_features (500, 78)
Y_train_binary__78_features (500,)
0.0    500
dtype: int64
Y_test_binary (500,)
classes_y:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
class_weights:  [1.14942529 0.95238095 1.07526882 0.85470085 0.98039216 1.07526882
 0.85470085 1.38888889 0.95238095 0.98039216 0.81300813 1.07526882
 1.38888889 0.92592593 0.9009009 ]

classes_class_weights: 
{0: 1.149,
 1: 0.952,
 2: 1.075,
 3: 0.855,
 4: 0.98,
 5: 1.075,
 6: 0.855,
 7: 1.389,
 8: 0.952,
 9: 0.98,
 10: 0.813,
 11: 1.075,
 12: 1.389,
 13: 0.926,
 14: 0.901}


c:\Users\androgo\anaconda3\envs\pytorch_env\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [4]:

import models

args = {
    'max_epochs' : 15,
    'no_trials': 15, 
    'trial_parameter':'accuracy', 
    'USE_AUTOMATIC_MIXED_PRECISION':True
}
study = optuna.create_study(direction = "maximize", sampler = optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())


args_model = {
    'input_features':1, 
    'number_of_classes':models_hyper_optim_optuna.number_of_classes, 
    'input_shape' : (32, models_hyper_optim_optuna.input_size, 78),
    'use_gru_instead_of_lstm':False
    }


cnn_birnn_model = models.CnnBirnnModel(args_model)
models_trainning = models_hyper_optim_optuna.neural_net_model_train.ModelsTrainning()

models_hyper_optim_optuna.start_optimize_objective(study, args,
                                                   cnn_birnn_model, models_trainning)



'''  
import time
start = time.time()
stop = time.time()
time_run= stop - start
print(time_run)
'''

[I 2022-09-24 21:43:51,181] A new study created in memory with name: no-name-9e0a569c-7b84-489d-9361-6a4412149b9f


NetModel(
  (cv1): Conv1d(1, 171, kernel_size=(3,), stride=(5,))
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(5,), padding=(0,))
  (bn1): BatchNorm1d(171, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(171, 50, kernel_size=(3,), stride=(4,))
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(4,), padding=(0,))
  (bn2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(50, 57, kernel_size=(1,), stride=(6,))
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(6,), padding=(0,))
  (bn3): BatchNorm1d(57, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (cv4): Conv1d(57, 209, kernel_size=(1,), stride=(2,))
  (avgp4): AvgPool1d(kernel_size=(1,), stride=(2,), padding=(0,))
  (bn4): BatchNorm1d(209, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel4): LeakyReLU

100%|██████████| 15/15 [00:02<00:00,  7.29it/s]


epoch 0 cost 2.064288 sec


100%|██████████| 15/15 [00:00<00:00, 24.74it/s]


...


Test set: Average loss: 0.0906, Accuracy: 0/500 (0%)

train loss:  3.2092799999999997
train accuracy:  micro
test loss:  tensor(0.0906)
test accuracy:  tensor(0.0750, device='cuda:0')
NetModel(
  (cv1): Conv1d(1, 155, kernel_size=(3,), stride=(6,))
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(6,), padding=(0,))
  (bn1): BatchNorm1d(155, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(155, 170, kernel_size=(1,), stride=(3,))
  (avgp2): AvgPool1d(kernel_size=(1,), stride=(3,), padding=(0,))
  (bn2): BatchNorm1d(170, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(170, 208, kernel_size=(1,), stride=(3,))
  (avgp3): AvgPool1d(kernel_size=(1,), stride=(3,), padding=(0,))
  (bn3): BatchNorm1d(208, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (bi_lstm): LSTM(208, 416, num_layers=2, 

100%|██████████| 15/15 [00:00<00:00, 38.80it/s]
[I 2022-09-24 21:44:57,699] Trial 13 pruned. 


NetModel(
  (cv1): Conv1d(1, 201, kernel_size=(3,), stride=(1,))
  (avgp1): AvgPool1d(kernel_size=(3,), stride=(1,), padding=(0,))
  (bn1): BatchNorm1d(201, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel1): LeakyReLU(negative_slope=0.01)
  (cv2): Conv1d(201, 101, kernel_size=(3,), stride=(2,))
  (avgp2): AvgPool1d(kernel_size=(3,), stride=(2,), padding=(0,))
  (bn2): BatchNorm1d(101, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel2): LeakyReLU(negative_slope=0.01)
  (cv3): Conv1d(101, 213, kernel_size=(3,), stride=(7,))
  (avgp3): AvgPool1d(kernel_size=(3,), stride=(7,), padding=(0,))
  (bn3): BatchNorm1d(213, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel3): LeakyReLU(negative_slope=0.01)
  (cv4): Conv1d(213, 131, kernel_size=(1,), stride=(6,))
  (avgp4): AvgPool1d(kernel_size=(1,), stride=(6,), padding=(0,))
  (bn4): BatchNorm1d(131, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lrel4): Lea

100%|██████████| 15/15 [00:00<00:00, 33.71it/s]
[I 2022-09-24 21:44:58,297] Trial 14 pruned. 


'  \nimport time\nstart = time.time()\nstop = time.time()\ntime_run= stop - start\nprint(time_run)\n'

# Tuning Visualization

###### graphs below are not visible in vs code but are in colab


In [5]:
import plotly
optuna.visualization.plot_intermediate_values(study)

In [6]:
optuna.visualization.plot_optimization_history(study)

In [7]:
optuna.visualization.plot_parallel_coordinate(study)

[W 2022-09-24 21:45:00,996] Your study has only completed trials with missing parameters.


In [8]:
optuna.visualization.plot_param_importances(study)